<center><img src="./logo.png" alt="Header" style="width: 800px;"/></center>

In [ ]:
import enum
import cv2

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [ ]:
import threading
import time
from Arm_Lib import Arm_Device


Arm = Arm_Device()
time.sleep(.1)

# 定义手势识别函数部分

# Define the gesture recognition function part

In [ ]:
import cv2 
import time
import demjson
import pygame 
from aip import AipBodyAnalysis
from aip import AipSpeech
from PIL import Image, ImageDraw, ImageFont
import numpy
import ipywidgets.widgets as widgets

# https://ai.baidu.com/ai-doc/BODY/4k3cpywrv
hand={'One':'数字1','Two':'数字2','Three':'数字3','Four':'数字4',
      'Five':'数字5', 'Six':'数字6','Seven':'数字7',
      'Eight':'数字8','Nine':'数字9','Fist':'拳头','Ok':'OK',
      'Prayer':'祈祷','Congratulation':'作揖','Honour':'作别',
      'Heart_single':'比心心','Thumb_up':'点赞','Thumb_down':'Diss',
      'ILY':'我爱你','Palm_up':'掌心向上','Heart_1':'双手比心1',
      'Heart_2':'双手比心2','Heart_3':'双手比心3','Rock':'Rock',
      'Insult':'竖中指','Face':'脸'}

# 下面的key要换成自己的  # Using your key and ID
""" APPID AK SK """
APP_ID = '24840713'
API_KEY = 'WMvYkrI7omD8OyGlwqjHQMRK'
SECRET_KEY = 'XAKxY0NyCL1QIfIRjEi8ScsPjGjsANuX'


client = AipBodyAnalysis(APP_ID, API_KEY, SECRET_KEY)


g_camera = cv2.VideoCapture(0)
g_camera.set(3, 640)
g_camera.set(4, 480)
g_camera.set(5, 30)  #设置帧率  #Set frame
g_camera.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
g_camera.set(cv2.CAP_PROP_BRIGHTNESS, 40) #设置亮度 -64 - 64  0.0  #set brightness -64 - 64  0.0
g_camera.set(cv2.CAP_PROP_CONTRAST, 50) #设置对比度 -64 - 64  2.0  #set contrast -64 - 64  2.0
g_camera.set(cv2.CAP_PROP_EXPOSURE, 156) #设置曝光值 1.0 - 5000  156.0  #set exposure 1.0 - 5000  156.0


ret, frame = g_camera.read()


# 定义摄像头显示组件

# Define camera widget

In [ ]:
image_widget = widgets.Image(format='jpeg', width=600, height=500) 
display(image_widget)   
image_widget.value = bgr8_to_jpeg(frame)

# 定义转换显示中文函数

# Define display Chinese text

In [ ]:
def cv2ImgAddText(img, text, left, top, textColor=(0, 255, 0), textSize=20):
    if (isinstance(img, numpy.ndarray)): 
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img)
    fontStyle = ImageFont.truetype(
        "simhei.ttf", textSize, encoding="utf-8")
    draw.text((left, top), text, textColor, font=fontStyle)
    return cv2.cvtColor(numpy.asarray(img), cv2.COLOR_RGB2BGR)

In [ ]:
look_at = [90, 164, 18, 0, 90, 90]
p_Prayer = [90, 90, 0, 180, 90, 180] 
p_Thumb_up = [90, 90, 90, 90, 90, 180] 
p_Heart_single = [90, 0, 180, 0, 90, 30] 

p_Eight = [90, 180, 18, 0, 90, 90] 

p_Congratulation = [90, 131, 52, 0, 90, 180] 
p_Rock = [90, 0, 90, 180, 90, 0] 
p_fist = [90, 90, 0, 0, 90, 0] 
p_horse_1 = [90, 7, 153, 19, 0, 126] 
p_horse_2 = [90, 5, 176, 0, 0, 180]
p_horse_3 = [90, 62, 158, 0, 0, 0]

global running
running = 0

In [ ]:
# Define control DOFBOT function, control No.1-No.5 servo，p=[S1,S2,S3,S4,S5]
# 定义移动机械臂函数,同时控制1-6号舵机运动，p=[S1,S2,S3,S4,S5,S6]
def arm_move_6(p, s_time = 500):
    for i in range(6):
        id = i + 1
        Arm.Arm_serial_servo_write(id, p[i], s_time)
        time.sleep(.01)
    time.sleep(s_time/1000)
    
# Define hourse movment    
# 定义小马运动
def horse_running():
    Arm.Arm_serial_servo_write(6, 150, 300)
    time.sleep(.3)
    Arm.Arm_serial_servo_write(6, 180, 300)
    time.sleep(.3)

In [ ]:
global g_state_arm
g_state_arm = 0
def ctrl_arm_move(index):
    global running
    if index == "Prayer":
        arm_move_6(p_Prayer, 1000)
        time.sleep(1.5)
        arm_move_6(look_at, 1000)
        time.sleep(1)
    elif index == "Thumb_up":
        s_time = 500
        Arm.Arm_serial_servo_write(6, 180, s_time)
        time.sleep(s_time/1000)
        Arm.Arm_serial_servo_write(6, 90, s_time)
        time.sleep(s_time/1000)
        Arm.Arm_serial_servo_write(6, 180, s_time)
        time.sleep(s_time/1000)
        Arm.Arm_serial_servo_write(6, 90, s_time)
        time.sleep(s_time/1000)
    elif index == "Ok":
        s_time = 300
        Arm.Arm_serial_servo_write(4, 10, s_time)
        time.sleep(s_time/1000)
        Arm.Arm_serial_servo_write(4, 0, s_time)
        time.sleep(s_time/1000)
        Arm.Arm_serial_servo_write(4, 10, s_time)
        time.sleep(s_time/1000)
        Arm.Arm_serial_servo_write(4, 0, s_time)
        time.sleep(s_time/1000)
    elif index == "Heart_single":
        arm_move_6([90, 90, 90, 90, 90, 90], 800)
        time.sleep(.1)
        arm_move_6(p_Heart_single, 1000)
        time.sleep(1)
    elif index == "Five":
        arm_move_6(look_at, 1000)
        time.sleep(.5)
    elif index == "Eight":
        s_time = 300
        arm_move_6(p_Eight, 0)
        time.sleep(1)
        Arm.Arm_serial_servo_write(2, 165, s_time)
        time.sleep(s_time/1000)
    elif index == "Rock":
        Arm.Arm_serial_servo_write6_array(p_Rock, 1300)
        time.sleep(3)
        Arm.Arm_serial_servo_write6_array(look_at, 1000)
        time.sleep(1)
    elif index == "Thumb_down": 
        Arm.Arm_serial_servo_write6_array(p_horse_1, 1300)
        time.sleep(1) 
    elif index == "Congratulation": 
        Arm.Arm_serial_servo_write6_array(p_horse_2, 1000)
        time.sleep(1)
        running = 1
        while running == 1:
            horse_running()
    elif index == "Seven": 
        Arm.Arm_Buzzer_On(8)   
        Arm.Arm_serial_servo_write6_array(p_horse_3, 1000)
        time.sleep(2)
        Arm.Arm_serial_servo_write6_array(look_at, 1000)
        time.sleep(1)
        

    global g_state_arm
    g_state_arm = 0
    

In [ ]:
arm_move_6(look_at, 1000)
time.sleep(1)

In [ ]:
def start_move_arm(index):
   
    global g_state_arm
    global running
    if g_state_arm == 0:
        closeTid = threading.Thread(target = ctrl_arm_move, args = [index])
        closeTid.setDaemon(True)
        closeTid.start()
        g_state_arm = 1
        
    if running == 1 and index == "Seven":
            running = 0

# 主进程

# Main process

In [ ]:
try:
    Arm.Arm_Buzzer_On(1)
    s_time = 300
    Arm.Arm_serial_servo_write(4, 10, s_time)
    time.sleep(s_time/1000)
    Arm.Arm_serial_servo_write(4, 0, s_time)
    time.sleep(s_time/1000)
    Arm.Arm_serial_servo_write(4, 10, s_time)
    time.sleep(s_time/1000)
    Arm.Arm_serial_servo_write(4, 0, s_time)
    time.sleep(s_time/1000)
    
    while True:
        ret, frame = g_camera.read()

        #image = get_file_content('./image.jpg')

        raw = str(client.gesture(image_widget.value))
        text = demjson.decode(raw)
        try:
            res = text['result'][0]['classname']
        except:
    #         img = cv2ImgAddText(frame, "unrecognized", 250, 30, (0, 0 , 255), 30)
            img = frame
        else:
    #         img = cv2ImgAddText(frame, hand[res], 250, 30, (0, 255 , 0), 30)
            if res == 'Prayer': 
                print('Recognition result：' + hand[res])
                img = cv2ImgAddText(frame, hand[res], 250, 30, (0, 255 , 0), 30)
                start_move_arm(res) 
            elif res == 'Thumb_up':
                print('Recognition result：' + hand[res])
                img = cv2ImgAddText(frame, hand[res], 250, 30, (0, 255 , 0), 30)
                start_move_arm(res)
            elif res == 'Ok': 
                print('Recognition result：' + hand[res])
                img = cv2ImgAddText(frame, hand[res], 250, 30, (0, 255 , 0), 30)
                start_move_arm(res)
            elif res == 'Heart_single': 
                print('Recognition result：' + hand[res])
                img = cv2ImgAddText(frame, hand[res], 250, 30, (0, 255 , 0), 30)
                start_move_arm(res)
            elif res == 'Five': 
                print('Recognition result：' + hand[res])
                img = cv2ImgAddText(frame, hand[res], 250, 30, (0, 255 , 0), 30)
                start_move_arm(res)
            elif res == "Eight": 
                print('Recognition result：' + hand[res])
                img = cv2ImgAddText(frame, hand[res], 250, 30, (0, 255 , 0), 30)
                start_move_arm(res)
                
            elif res == "Rock": 
                print('Recognition result：' + hand[res])
                img = cv2ImgAddText(frame, hand[res], 250, 30, (0, 255 , 0), 30)
                start_move_arm(res)
            elif res == "Congratulation": 
                print('Recognition result：' + hand[res])
                img = cv2ImgAddText(frame, hand[res], 250, 30, (0, 255 , 0), 30)
                start_move_arm(res)
            elif res == "Seven": 
                print('Recognition result：' + hand[res])
                img = cv2ImgAddText(frame, hand[res], 250, 30, (0, 255 , 0), 30)
                start_move_arm(res)
            elif res == "Thumb_down": 
                print('Recognition result：' + hand[res])
                img = cv2ImgAddText(frame, hand[res], 250, 30, (0, 255 , 0), 30)
                start_move_arm(res)    
                
            else:
                img = frame

        image_widget.value = bgr8_to_jpeg(img)
except KeyboardInterrupt:
    print(" Program closed! ")
    pass


In [ ]:
g_camera.release()             